Tugas 1

In [1]:
import findspark
findspark.init()

from pyspark.sql import SparkSession

# Inisialisasi SparkSession
spark = SparkSession.builder.appName('TugasPertemuan6').getOrCreate()

# Data untuk DataFrame yang lebih besar
data = [
    ('Andi', 'Teknik Informatika', 85),
    ('Budi', 'Sistem Informasi', 90),
    ('Citra', 'Teknik Informatika', 78),
    ('Dewi', 'Sistem Informasi', 92),
    ('Eko', 'Teknik Elektro', 88),
    ('Fani', 'Teknik Elektro', 76),
    ('Gina', 'Sistem Informasi', 95),
    ('Hadi', 'Teknik Informatika', 83),
    ('Ira', 'Teknik Elektro', 80),
    ('Joko', 'Sistem Informasi', 89),
    ('Kiki', 'Teknik Informatika', 91),
    ('Lina', 'Teknik Elektro', 85)
]

# Skema kolom
columns = ['NamaSiswa', 'Jurusan', 'NilaiAkhir']

# Membuat DataFrame
df = spark.createDataFrame(data, schema=columns)

# Menampilkan DataFrame
df.show()

+---------+------------------+----------+
|NamaSiswa|           Jurusan|NilaiAkhir|
+---------+------------------+----------+
|     Andi|Teknik Informatika|        85|
|     Budi|  Sistem Informasi|        90|
|    Citra|Teknik Informatika|        78|
|     Dewi|  Sistem Informasi|        92|
|      Eko|    Teknik Elektro|        88|
|     Fani|    Teknik Elektro|        76|
|     Gina|  Sistem Informasi|        95|
|     Hadi|Teknik Informatika|        83|
|      Ira|    Teknik Elektro|        80|
|     Joko|  Sistem Informasi|        89|
|     Kiki|Teknik Informatika|        91|
|     Lina|    Teknik Elektro|        85|
+---------+------------------+----------+



Tugas 2

In [2]:
# Filter siswa dengan nilai lebih dari 85
print("Siswa dengan Nilai Akhir > 85:")
df.filter(df["NilaiAkhir"] > 85).show()

# Select kolom NamaSiswa dan NilaiAkhir
print(" Nama Siswa dan Nilai Akhir:")
df.select("NamaSiswa", "NilaiAkhir").show()

# GroupBy berdasarkan Jurusan dan menghitung rata-rata Nilai Akhir
print("Rata-rata Nilai Akhir per Jurusan:")
df.groupBy("Jurusan").mean("NilaiAkhir").show()

# GroupBy berdasarkan Jurusan dan mencari nilai tertinggi
print("Nilai Tertinggi per Jurusan:")
df.groupBy("Jurusan").max("NilaiAkhir").show()

# GroupBy berdasarkan Jurusan dan menghitung total Nilai Akhir
print("Total Nilai Akhir per Jurusan:")
df.groupBy("Jurusan").sum("NilaiAkhir").show()

#  Menggabungkan mean, max, dan sum dalam satu operasi groupBy
print("Rata-rata, Nilai Maksimum, dan Total Nilai Akhir per Jurusan:")
df.groupBy("Jurusan").agg(
    {"NilaiAkhir": "mean", "NilaiAkhir": "max", "NilaiAkhir": "sum"}
).show()

Siswa dengan Nilai Akhir > 85:
+---------+------------------+----------+
|NamaSiswa|           Jurusan|NilaiAkhir|
+---------+------------------+----------+
|     Budi|  Sistem Informasi|        90|
|     Dewi|  Sistem Informasi|        92|
|      Eko|    Teknik Elektro|        88|
|     Gina|  Sistem Informasi|        95|
|     Joko|  Sistem Informasi|        89|
|     Kiki|Teknik Informatika|        91|
+---------+------------------+----------+

 Nama Siswa dan Nilai Akhir:
+---------+----------+
|NamaSiswa|NilaiAkhir|
+---------+----------+
|     Andi|        85|
|     Budi|        90|
|    Citra|        78|
|     Dewi|        92|
|      Eko|        88|
|     Fani|        76|
|     Gina|        95|
|     Hadi|        83|
|      Ira|        80|
|     Joko|        89|
|     Kiki|        91|
|     Lina|        85|
+---------+----------+

Rata-rata Nilai Akhir per Jurusan:
+------------------+---------------+
|           Jurusan|avg(NilaiAkhir)|
+------------------+---------------+
|Tek

Tugas 3

In [3]:
# Menambahkan kolom bonus nilai (10% dari NilaiAkhir)
df = df.withColumn('BonusNilai', df['NilaiAkhir'] * 0.1)

# Menambahkan kolom total nilai (NilaiAkhir + BonusNilai)
df = df.withColumn('TotalNilai', df['NilaiAkhir'] + df['BonusNilai'])

# Menampilkan DataFrame dengan kolom baru
df.show()

+---------+------------------+----------+------------------+----------+
|NamaSiswa|           Jurusan|NilaiAkhir|        BonusNilai|TotalNilai|
+---------+------------------+----------+------------------+----------+
|     Andi|Teknik Informatika|        85|               8.5|      93.5|
|     Budi|  Sistem Informasi|        90|               9.0|      99.0|
|    Citra|Teknik Informatika|        78| 7.800000000000001|      85.8|
|     Dewi|  Sistem Informasi|        92| 9.200000000000001|     101.2|
|      Eko|    Teknik Elektro|        88|               8.8|      96.8|
|     Fani|    Teknik Elektro|        76|7.6000000000000005|      83.6|
|     Gina|  Sistem Informasi|        95|               9.5|     104.5|
|     Hadi|Teknik Informatika|        83|               8.3|      91.3|
|      Ira|    Teknik Elektro|        80|               8.0|      88.0|
|     Joko|  Sistem Informasi|        89|               8.9|      97.9|
|     Kiki|Teknik Informatika|        91|               9.1|    

Tugas 4

In [4]:
from pyspark.sql.window import Window
from pyspark.sql import functions as F

# Definisikan window berdasarkan partisi 'Jurusan' dan urutkan berdasarkan 'NilaiAkhir'
windowSpec = Window.partitionBy("Jurusan").orderBy("NilaiAkhir")

# Hitung running total (jumlah kumulatif) dari NilaiAkhir
df = df.withColumn("RunningTotal", F.sum("NilaiAkhir").over(windowSpec))

# Menampilkan DataFrame dengan kolom RunningTotal
df.show()

+---------+------------------+----------+------------------+----------+------------+
|NamaSiswa|           Jurusan|NilaiAkhir|        BonusNilai|TotalNilai|RunningTotal|
+---------+------------------+----------+------------------+----------+------------+
|     Joko|  Sistem Informasi|        89|               8.9|      97.9|          89|
|     Budi|  Sistem Informasi|        90|               9.0|      99.0|         179|
|     Dewi|  Sistem Informasi|        92| 9.200000000000001|     101.2|         271|
|     Gina|  Sistem Informasi|        95|               9.5|     104.5|         366|
|     Fani|    Teknik Elektro|        76|7.6000000000000005|      83.6|          76|
|      Ira|    Teknik Elektro|        80|               8.0|      88.0|         156|
|     Lina|    Teknik Elektro|        85|               8.5|      93.5|         241|
|      Eko|    Teknik Elektro|        88|               8.8|      96.8|         329|
|    Citra|Teknik Informatika|        78| 7.800000000000001|     